In [1]:
import pandas as pd
from pathlib import Path

from matplotlib import pyplot

from political_party_analysis.loader import DataLoader
from political_party_analysis.visualization import scatter_plot

In [6]:
Party| description of dataset 

,country,eastwest,party,party_id,eu_position,eu_position_sd,eu_salience,eu_dissent,eu_blur,eu_cohesion,...,regions,russian_interference,anti_islam_rhetoric,people_vs_elite,antielite_salience,corrupt_salience,members_vs_leadership,eu_econ_require,eu_political_require,eu_googov_require
0,be,1.0,PS,102.0,6.083333,0.288675,4.272727,2.750000,2.166667,6.250,...,6.000000,0.111111,2.111111,3.363636,2.181818,1.909091,7.500000,NaN,NaN,NaN
1,be,1.0,SP/SPA,103.0,6.000000,0.426401,4.181818,2.500000,1.714286,6.000,...,5.333333,0.111111,2.222222,3.545455,2.090909,1.636364,6.818182,NaN,NaN,NaN
2,be,1.0,ECOLO,104.0,6.583333,0.514929,4.818182,2.500000,1.428571,6.000,...,6.166667,0.111111,2.888889,5.636364,2.454545,2.363636,3.909091,NaN,NaN,NaN
3,be,1.0,AGALEV,105.0,6.583333,0.514929,4.818182,2.500000,1.285714,5.875,...,6.083333,0.111111,2.777778,5.636364,2.363636,2.000000,4.000000,NaN,NaN,NaN
4,be,1.0,PRL/MR,106.0,6.500000,0.674200,5.090909,2.000000,1.714286,5.500,...,6.416667,0.111111,2.333333,4.636364,1.727273,1.636364,7.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,ice,NaN,M,4504.0,1.200000,0.447214,5.600000,2.000000,0.500000,NaN,...,4.000000,0.800000,2.000000,3.200000,4.400000,1.500000,9.400000,NaN,NaN,NaN
273,ice,NaN,Sam,4505.0,6.400000,0.547723,6.800000,2.333333,4.000000,NaN,...,6.000000,0.000000,0.000000,5.000000,1.400000,5.800000,6.400000,NaN,NaN,NaN
274,ice,NaN,Pi,4506.0,4.800000,0.447214,4.600000,3.333333,4.500000,NaN,...,6.000000,0.000000,0.000000,5.800000,6.200000,8.400000,2.800000,NaN,NaN,NaN
275,ice,NaN,V,4507.0,6.600000,0.547723,8.000000,2.000000,2.500000,NaN,...,6.000000,0.000000,0.000000,3.600000,0.200000,4.600000,7.400000,NaN,NaN,NaN


In [4]:
df.columns

Index(['country', 'eastwest', 'party', 'party_id', 'eu_position',
       'eu_position_sd', 'eu_salience', 'eu_dissent', 'eu_blur', 'eu_cohesion',
       'eu_foreign', 'eu_intmark', 'eu_budgets', 'eu_asylum', 'lrgen',
       'lrecon', 'lrecon_sd', 'lrecon_salience', 'lrecon_dissent',
       'lrecon_blur', 'galtan', 'galtan_sd', 'galtan_salience',
       'galtan_dissent', 'galtan_blur', 'immigrate_policy',
       'immigrate_salience', 'immigrate_dissent', 'multiculturalism',
       'multicult_salience', 'multicult_dissent', 'redistribution',
       'redist_salience', 'environment', 'enviro_salience', 'spendvtax',
       'deregulation', 'econ_interven', 'civlib_laworder', 'sociallifestyle',
       'religious_principles', 'ethnic_minorities', 'nationalism',
       'urban_rural', 'protectionism', 'regions', 'russian_interference',
       'anti_islam_rhetoric', 'people_vs_elite', 'antielite_salience',
       'corrupt_salience', 'members_vs_leadership', 'eu_econ_require',
       'eu_political

In [ ]:
# Step 1: loading and preprocessing data

# Step 2: Model training

# Step 3: Model registering

# Step 4: Model deployment

In [ ]:
# Dimensionality reduction step
##### YOUR CODE GOES HERE #####

## Uncomment this snippet to plot dim reduced data
# pyplot.figure()
# splot = pyplot.subplot()
# scatter_plot(
#     reduced_dim_data,
#     color="r",
#     splot=splot,
#     label="dim reduced data",
# )
# pyplot.savefig(Path(__file__).parents[1].joinpath(*["plots", "dim_reduced_data.png"]))

# Density estimation/distribution modelling step
##### YOUR CODE GOES HERE #####

# Plot density estimation results here
##### YOUR CODE GOES HERE #####
pyplot.savefig(Path(__file__).parents[1].joinpath(*["plots", "density_estimation.png"]))

# Plot left and right wing parties here
pyplot.figure()
splot = pyplot.subplot()
##### YOUR CODE GOES HERE #####
pyplot.savefig(Path(__file__).parents[1].joinpath(*["plots", "left_right_parties.png"]))
pyplot.title("Lefty/righty parties")

# Plot finnish parties here
##### YOUR CODE GOES HERE #####

print("Analysis Complete")
